In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
train=pd.read_csv('Train.csv', delimiter=',')

In [3]:
test=pd.read_csv('Test.csv', delimiter=',')

In [4]:
train.head()

,id,NewsText,label
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1
1,1059,Karanta cikakken labarin a nan,0
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0


In [5]:
#for cleaning the newstext colum
def data_remove(df, remove, add):
    x_list = []
    
    for i, df_str in enumerate(df):
        if remove in df_str:
            new = df_str.replace(remove, add)
            x_list.append(new)
            print(f'index:{i} Old Data: {df_str}')
            print(f'New Data: {new}')
        else:
            x_list.append(df_str)
            
    return x_list

In [6]:
train['NewsText'] = data_remove(train['NewsText'],remove = '₦', add = '' )

index:57 Old Data: Bankin Duniya zai tallafa wa Najeriya da biliyan ₦150 don haɓaka lantarki
New Data: Bankin Duniya zai tallafa wa Najeriya da biliyan 150 don haɓaka lantarki


In [7]:
train['NewsText'] = data_remove(train['NewsText'],remove = ')', add = '' )

index:78 Old Data:  Don amfani da Bitcoins, matakin farko shi ne bude abin da ake kira 'wallet' ko lalita (wadda ke iya zama a intanet ko a manhajar waya ko a wata na'ura mai tsaro)
New Data:  Don amfani da Bitcoins, matakin farko shi ne bude abin da ake kira 'wallet' ko lalita (wadda ke iya zama a intanet ko a manhajar waya ko a wata na'ura mai tsaro


In [8]:
train['NewsText'] = data_remove(train['NewsText'],remove = '(', add = '' )

index:78 Old Data:  Don amfani da Bitcoins, matakin farko shi ne bude abin da ake kira 'wallet' ko lalita (wadda ke iya zama a intanet ko a manhajar waya ko a wata na'ura mai tsaro
New Data:  Don amfani da Bitcoins, matakin farko shi ne bude abin da ake kira 'wallet' ko lalita wadda ke iya zama a intanet ko a manhajar waya ko a wata na'ura mai tsaro


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold

In [62]:
cv = CountVectorizer()
tv = TfidfVectorizer(token_pattern='(?u)\\b\\w+\\b')
tfd = TfidfTransformer()

In [11]:
arr = np.array(train['NewsText'][0], dtype = 'object')

In [12]:
#tfd.fit_transform(tv.fit_transform(train['NewsText']).toarray()).toarray()

tfd_data = tv.fit(train['NewsText'])

In [13]:
words = tfd_data.get_feature_names()

In [14]:
len(words)

1416

In [15]:
np.array(words[-20:])

array(['ƙuri', 'ƙwararan', 'ƴan', 'ƴar', 'ƴaƴana', 'ɓalle', 'ɓangare',
       'ɓullar', 'ɓullarta', 'ɓullo', 'ɗaki', 'ɗalibai', 'ɗan', 'ɗare',
       'ɗaruruwan', 'ɗaukar', 'ɗauke', 'ɗaya', 'ɗayan', 'ɗokin'],
      dtype='<U9')

<b> Since hausa dataset do not have a stop word or i dont know of any so i could not try to create stop word feature list and work on some technique</b>

## Modelling

In [16]:
def transform(data, details):
    # 0 == -1 , 1 == 0, 2 == 1
    
    x = []
    if details == 'encode':
        for v, i in enumerate(data):
            if i == -1: x.append(0)
            elif i == 0: x.append(1)
            else: x.append(2)
    else:
        for v, i in enumerate(data):
            if i == 0: x.append(-1)
            elif i == 1: x.append(0)
            else: x.append(1)
                           
    return x

In [17]:
y = train['label'].values

In [18]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold,cross_val_predict, RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV, RandomizedSearchCV

In [19]:
str_kf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 1) 
skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

In [20]:
def model_func(data, tar):
    models = [RandomForestClassifier, LogisticRegression, BernoulliNB, MultinomialNB, XGBClassifier]

    for model in models:
        scores = cross_val_score(estimator=model(), X=data, y=tar, cv=skf, n_jobs=-1, scoring='accuracy')
        print(model.__name__, '---- Validation Results --->' , scores.mean())
        print('______')

In [21]:
#TfidfVectorizer

model_func(data = tv.fit_transform(train['NewsText']), tar = transform(y, details = 'encode'))

RandomForestClassifier ---- Validation Results ---> 0.5555555555555555
______
LogisticRegression ---- Validation Results ---> 0.611111111111111
______
BernoulliNB ---- Validation Results ---> 0.5888888888888888
______
MultinomialNB ---- Validation Results ---> 0.6180555555555555
______


KeyboardInterrupt: 

In [25]:
#TfidfTransformer

model_func(data = tfd.fit_transform(tv.fit_transform(train['NewsText'])), tar = transform(y, details = 'encode'))

RandomForestClassifier ---- Validation Results ---> 0.548611111111111
______
LogisticRegression ---- Validation Results ---> 0.6013888888888891
______
BernoulliNB ---- Validation Results ---> 0.5888888888888888
______
MultinomialNB ---- Validation Results ---> 0.6180555555555556
______


KeyboardInterrupt: 

In [26]:
mul = MultinomialNB()
log = LogisticRegression()


In [63]:
data_tv = tv.fit_transform(train['NewsText']).toarray()
data_tfd = tfd.fit_transform(data_tv).toarray()
target = transform(y, details = 'encode')

cv_train = cv.fit_transform(train['NewsText']).toarray()

In [ ]:
cv_results = cross_validate (log, data_tv, target, cv = skf, scoring = 'accuracy', return_train_score=True)

In [ ]:
print(f"Train Score: {cv_results['train_score'].mean()}\n")
print(f"Test Score: {cv_results['test_score'].mean()}")

In [64]:
## Hyper Parameter tuning for logistic regression model
start = time.time()

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l1', 'l2']
c_values = [1000, 500, 200, 100, 10, 1.0, 0.1, 0.01, 0.001, 0.0001]
max_iter = [i*100 for i in range(1,11)]
multi_class = ['auto', 'ovr', 'multinomial']

#define grid serch 
grid = dict(solver = solvers, penalty = penalty, C = c_values)
grid_search = GridSearchCV(estimator = model, param_grid = grid, scoring = 'accuracy', n_jobs = -1, cv = skf)
grid_result = grid_search.fit(cv_train, target)

#summarize results
print(f"Best grid score: {grid_result.best_score_}")
print(f"Best grid params: {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean , stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
end = time.time()
print(f'Time Spent: {end - start}')

Best grid score: 0.6055555555555555
Best grid params: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
nan (nan) with: {'C': 1000, 'penalty': 'l1', 'solver': 'newton-cg'}
nan (nan) with: {'C': 1000, 'penalty': 'l1', 'solver': 'lbfgs'}
0.511111 (0.064490) with: {'C': 1000, 'penalty': 'l1', 'solver': 'liblinear'}
0.572222 (0.057869) with: {'C': 1000, 'penalty': 'l2', 'solver': 'newton-cg'}
0.573611 (0.058860) with: {'C': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
0.581944 (0.059641) with: {'C': 1000, 'penalty': 'l2', 'solver': 'liblinear'}
nan (nan) with: {'C': 500, 'penalty': 'l1', 'solver': 'newton-cg'}
nan (nan) with: {'C': 500, 'penalty': 'l1', 'solver': 'lbfgs'}
0.529167 (0.055277) with: {'C': 500, 'penalty': 'l1', 'solver': 'liblinear'}
0.572222 (0.058366) with: {'C': 500, 'penalty': 'l2', 'solver': 'newton-cg'}
0.572222 (0.058366) with: {'C': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
0.581944 (0.060604) with: {'C': 500, 'penalty': 'l2', 'solver': 'liblinear'}
nan (nan) with: {'C':

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.51111111 0.57222222 0.57361111 0.58194444
        nan        nan 0.52916667 0.57222222 0.57222222 0.58194444
        nan        nan 0.525      0.57638889 0.57638889 0.58194444
        nan        nan 0.52361111 0.57777778 0.57777778 0.58194444
        nan        nan 0.52638889 0.57361111 0.57361111 0.58333333
        nan        nan 0.58055556 0.58472222 0.58472222 0.59722222
        nan        nan 0.4        0.6        0.6        0.60555556
        nan        nan 0.34166667 0.53333333 0.53333333 0.54444444
        nan        nan 0.34166667 0.36944444 0.36944444 0.44305556
        nan        nan 0.34166667 0.36666667 0.36666667 0.425     ]
  warnings.warn(


In [65]:
grid_search.best_estimator_

LogisticRegression(C=0.1, solver='liblinear')

In [66]:
grid_search.best_score_

0.6055555555555555

In [67]:
grid_search.best_params_

{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}

In [84]:
tune_log = LogisticRegression(C = 0.1, penalty = 'l2', solver = 'liblinear')

In [85]:
from sklearn.metrics import accuracy_score

In [86]:
cv_score=[]
train_score=[]
sub_pred = []

fold= skf
X = pd.DataFrame(cv_train)
yy = target
testing = cv.transform(test['NewsText']).toarray()


for train_index, test_index in fold.split(X,pd.Series(yy)):
    X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
    y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]
    
    tune_log.fit(X_trainx,y_trainx)
    train_preds=tune_log.predict(X_trainx)
    print("Train Accuracy: ",accuracy_score(y_trainx,train_preds))
    train_score.append(accuracy_score(y_trainx,train_preds))
    
    val_preds=tune_log.predict(X_val)
    print("Validation Accuracy: ",accuracy_score(y_val,val_preds))
    cv_score.append(accuracy_score(y_val,val_preds))
    
    tested = tune_log.predict_proba(testing)
    print("Test has been Scored")
    print("______________\n")
    sub_pred.append(tested)

print(f'Training Mean ROC AUC Score {np.mean(train_score)}')
print(f'Validation Mean ROC AUC Score {np.mean(cv_score)}')

Train Accuracy:  0.9479166666666666
Validation Accuracy:  0.5833333333333334
Test has been Scored
______________

Train Accuracy:  0.9635416666666666
Validation Accuracy:  0.6458333333333334
Test has been Scored
______________

Train Accuracy:  0.9635416666666666
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.9583333333333334
Validation Accuracy:  0.7083333333333334
Test has been Scored
______________

Train Accuracy:  0.9635416666666666
Validation Accuracy:  0.6041666666666666
Test has been Scored
______________

Train Accuracy:  0.953125
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.953125
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.9635416666666666
Validation Accuracy:  0.625
Test has been Scored
______________

Train Accuracy:  0.9635416666666666
Validation Accuracy:  0.6875
Test has been Scored
______________

Train Accuracy:  0.953125
Validation Accuracy:  0.52083

In [87]:
value_log = (np.sum(sub_pred, axis = 0))/15

In [88]:
val_log = np.argmax(value, axis = 1)

In [89]:
labelss = transform(data = val_log, details = 'decode')

In [90]:
sub = pd.DataFrame({'id':test['id'], 'label':labelss})

In [92]:
sub.label.value_counts()

 1    26
-1    23
 0    11
Name: label, dtype: int64

In [94]:
sub.to_csv('log_reg_model.csv', index = False)

In [95]:
tune_log = RandomForestClassifier(max_depth=8, max_features='sqrt', min_samples_leaf=2,min_samples_split=10, n_estimators = 60)

In [96]:
cv_score=[]
train_score=[]
sub_pred = []

fold= skf
X = pd.DataFrame(cv_train)
yy = target
testing = cv.transform(test['NewsText']).toarray()


for train_index, test_index in fold.split(X,pd.Series(yy)):
    X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
    y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]
    
    tune_log.fit(X_trainx,y_trainx)
    train_preds=tune_log.predict(X_trainx)
    print("Train Accuracy: ",accuracy_score(y_trainx,train_preds))
    train_score.append(accuracy_score(y_trainx,train_preds))
    
    val_preds=tune_log.predict(X_val)
    print("Validation Accuracy: ",accuracy_score(y_val,val_preds))
    cv_score.append(accuracy_score(y_val,val_preds))
    
    tested = tune_log.predict_proba(testing)
    print("Test has been Scored")
    print("______________\n")
    sub_pred.append(tested)

print(f'Training Mean ROC AUC Score {np.mean(train_score)}')
print(f'Validation Mean ROC AUC Score {np.mean(cv_score)}')

Train Accuracy:  0.7864583333333334
Validation Accuracy:  0.5416666666666666
Test has been Scored
______________

Train Accuracy:  0.8177083333333334
Validation Accuracy:  0.5416666666666666
Test has been Scored
______________

Train Accuracy:  0.78125
Validation Accuracy:  0.5
Test has been Scored
______________

Train Accuracy:  0.8229166666666666
Validation Accuracy:  0.6875
Test has been Scored
______________

Train Accuracy:  0.796875
Validation Accuracy:  0.5416666666666666
Test has been Scored
______________

Train Accuracy:  0.7708333333333334
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.8125
Validation Accuracy:  0.6875
Test has been Scored
______________

Train Accuracy:  0.7916666666666666
Validation Accuracy:  0.625
Test has been Scored
______________

Train Accuracy:  0.7447916666666666
Validation Accuracy:  0.5833333333333334
Test has been Scored
______________

Train Accuracy:  0.8072916666666666
Validation Accuracy:  0.60416666666

In [98]:
value_rfc = (np.sum(sub_pred, axis = 0))/15

In [99]:
val_rfc = np.argmax(value_rfc*0.6 + value_log*0.4, axis = 1)

In [100]:
val_rfc

array([2, 2, 1, 0, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 0,
       0, 0, 2, 2, 1, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 2,
       2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 2, 2, 0, 2, 0, 0], dtype=int64)

In [101]:
labelss = transform(data = val_rfc, details = 'decode')
sub = pd.DataFrame({'id':test['id'], 'label':labelss})
sub.label.value_counts()

 1    35
-1    20
 0     5
Name: label, dtype: int64

In [102]:
sub.to_csv('ensemble.csv', index = False)

## Multinomial NB

In [46]:
cv_score=[]
train_score=[]
sub_pred = []

fold= skf
X = pd.DataFrame(data_tv)
yy = target
testing = tv.transform(test['NewsText']).toarray()


for train_index, test_index in fold.split(X,pd.Series(yy)):
    X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
    y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]
    
    mul.fit(X_trainx,y_trainx)
    train_preds=tune_log.predict(X_trainx)
    print("Train Accuracy: ",accuracy_score(y_trainx,train_preds))
    train_score.append(accuracy_score(y_trainx,train_preds))
    
    val_preds=mul.predict(X_val)
    print("Validation Accuracy: ",accuracy_score(y_val,val_preds))
    cv_score.append(accuracy_score(y_val,val_preds))
    
    tested = mul.predict_proba(testing)
    print("Test has been Scored")
    print("______________\n")
    sub_pred.append(tested)

print(f'Training Mean ROC AUC Score {np.mean(train_score)}')
print(f'Validation Mean ROC AUC Score {np.mean(cv_score)}')

Train Accuracy:  0.921875
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.9479166666666666
Validation Accuracy:  0.6041666666666666
Test has been Scored
______________

Train Accuracy:  0.9479166666666666
Validation Accuracy:  0.5833333333333334
Test has been Scored
______________

Train Accuracy:  0.9270833333333334
Validation Accuracy:  0.6458333333333334
Test has been Scored
______________

Train Accuracy:  0.921875
Validation Accuracy:  0.6458333333333334
Test has been Scored
______________

Train Accuracy:  0.9322916666666666
Validation Accuracy:  0.5625
Test has been Scored
______________

Train Accuracy:  0.9270833333333334
Validation Accuracy:  0.7083333333333334
Test has been Scored
______________

Train Accuracy:  0.9322916666666666
Validation Accuracy:  0.6875
Test has been Scored
______________

Train Accuracy:  0.9270833333333334
Validation Accuracy:  0.6458333333333334
Test has been Scored
______________

Train Accuracy:  0.94791666666

In [47]:
value = (np.sum(sub_pred, axis = 0))/15
val = np.argmax(value, axis = 1)

In [48]:
labels = transform(data = val, details = 'decode')

In [49]:
sub_mul = pd.DataFrame({'id':test['id'], 'label':labels})

In [50]:
sub_mul.label.value_counts()

 1    30
-1    23
 0     7
Name: label, dtype: int64

In [51]:
sub.to_csv('multinomialnb_model.csv', index = False)